In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
# spark = SparkSession.builder.getOrCreate()
spark = get_spark_session(dev=True)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 06:24:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}")

DataFrame[]

In [3]:
geolocation_df = spark.read.csv("s3a://warehousedev/bronze/tsv/geolocation.tsv", header=True, sep='\t')
geolocation_df.show(n=5)

25/09/23 06:24:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------+-------------------+-------------------+-----+---------+
|zip_code|                lat|                lng|state|     city|
+--------+-------------------+-------------------+-----+---------+
|    1001|-23.550189776551765|  -46.6340235559042|   SP|sao paulo|
|    1002| -23.54814573176355| -46.63497921074498|   SP|sao paulo|
|    1003| -23.54899372481316|-46.635731309975874|   SP|sao paulo|
|    1004|-23.549798842277006| -46.63475694378971|   SP|sao paulo|
|    1005|-23.549456199830747| -46.63673294803691|   SP|sao paulo|
+--------+-------------------+-------------------+-----+---------+
only showing top 5 rows



In [4]:
geolocation_df.count()

19015

In [5]:
lat_min, lat_max = -33.75, 5.27
lng_min, lng_max = -73.98, -34.79

# 2. filter()를 사용해 경계 내의 데이터만 추출
condtion = (F.col('lat') >= lat_min) & (F.col('lat') <= lat_max) & (F.col('lng') >= lng_min) & (F.col('lng') <= lng_max)
brazil_df = geolocation_df.filter(condtion)

print("▼▼▼ 브라질 경계 내의 데이터만 필터링한 결과 ▼▼▼")
brazil_df.show()

▼▼▼ 브라질 경계 내의 데이터만 필터링한 결과 ▼▼▼
+--------+-------------------+-------------------+-----+---------+
|zip_code|                lat|                lng|state|     city|
+--------+-------------------+-------------------+-----+---------+
|    1001|-23.550189776551765|  -46.6340235559042|   SP|sao paulo|
|    1002| -23.54814573176355| -46.63497921074498|   SP|sao paulo|
|    1003| -23.54899372481316|-46.635731309975874|   SP|sao paulo|
|    1004|-23.549798842277006| -46.63475694378971|   SP|sao paulo|
|    1005|-23.549456199830747| -46.63673294803691|   SP|sao paulo|
|    1006| -23.55010181145287|-46.636136735174176|   SP|sao paulo|
|    1007|-23.550046202351666|-46.637251459637646|   SP|sao paulo|
|    1008| -23.54600174938346| -46.63588592135778|   SP|sao paulo|
|    1009|-23.546835208320868|-46.636490695312204|   SP|sao paulo|
|    1010| -23.54638943836927|-46.635226081509046|   SP|sao paulo|
|    1011| -23.54690864025111| -46.63558202636042|   SP|sao paulo|
|    1012|-23.547789733146864| 

In [6]:
not_brazil = geolocation_df.filter(~condtion)
not_brazil.show(truncate=False)

+--------+--------------------+-------------------+-----+-----------------------+
|zip_code|lat                 |lng                |state|city                   |
+--------+--------------------+-------------------+-----+-----------------------+
|18243   |28.008978338034268  |-15.536867177679133|SP   |bom retiro da esperanca|
|28155   |-11.314698067779764 |-34.725129839453665|RJ   |santa maria            |
|46560   |13.001420173555276  |-23.583939075194326|BA   |ibiajara               |
|53990   |-3.848720972616623  |-32.41410011794933 |PE   |fernando de noronha    |
|57319   |-0.48350469590587863|-28.424444123004108|AL   |vila sao francisco     |
|58441   |5.0135194807895935  |-29.384845587162516|PB   |são josé da mata       |
|68275   |22.56795175383837   |-27.69808676596982 |PA   |porto trombetas        |
|68379   |9.049062717536653   |-25.170903455030523|PA   |castelo dos sonhos     |
|68447   |7.271649375032848   |-40.315884016386505|PA   |barcarena              |
|78131   |38.816

In [7]:
customer_df = spark.read.csv("s3a://warehousedev/bronze/tsv/customer.tsv", header=True, sep='\t')
customer_df = customer_df.withColumn('user_type', F.lit('customer')).withColumnRenamed('customer_id', 'user_id')
seller_df = spark.read.csv("s3a://warehousedev/bronze/tsv/seller.tsv", header=True, sep='\t')
seller_df = seller_df.withColumn('user_type', F.lit('seller')).withColumnRenamed('seller_id', 'user_id')

In [8]:
all_user_df = customer_df.union(seller_df).dropDuplicates()

In [9]:
user_location = all_user_df.join(geolocation_df, on='zip_code', how='inner')
user_location = user_location.orderBy('zip_code')

In [10]:
user_location.writeTo(f"{test_namespace}.user_location").using('iceberg').createOrReplace()

In [11]:
spark.stop()